In [ ]:


import pandas as pd

from os.path import expanduser

SRC_PATH = expanduser("~") + '/congenial-broccoli/boston-housing/'
print(SRC_PATH)
print(SRC_PATH)

In [1]:
import pyspark
from pyspark.context import SparkContext

sc = SparkContext('local', 'test')


ModuleNotFoundError: No module named 'pyspark'

In [2]:
from pyspark.sql import SQLContext

sql = SQLContext(sc)


ModuleNotFoundError: No module named 'pyspark'

In [3]:
housing_df = sql.read.csv(SRC_PATH + 'train.csv', header=True, inferSchema=True)


NameError: name 'sql' is not defined

In [4]:
housing_df.show(4)


NameError: name 'housing_df' is not defined

In [5]:
from pyspark.ml.feature import VectorAssembler

training_features = ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'tax', 'ptratio', 'lstat']
vector_assembler = VectorAssembler(inputCols=training_features, outputCol="features")


ModuleNotFoundError: No module named 'pyspark'

In [6]:
df_with_features_vector = vector_assembler.transform(housing_df)
df_with_features_vector.show(3)


NameError: name 'vector_assembler' is not defined

In [7]:
train_df, test_df = df_with_features_vector.randomSplit([0.8, 0.2], seed=17)


NameError: name 'df_with_features_vector' is not defined

In [8]:
from pyspark.ml.regression import LinearRegression

linear = LinearRegression(featuresCol="features", labelCol="medv")
linear_model = linear.fit(train_df)


ModuleNotFoundError: No module named 'pyspark'

In [9]:
predictions_df = linear_model.transform(test_df)
predictions_df.show(3)


NameError: name 'linear_model' is not defined

In [10]:
linear_model.summary.r2


NameError: name 'linear_model' is not defined

In [11]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="r2")
evaluator.evaluate(predictions_df)


ModuleNotFoundError: No module named 'pyspark'

In [12]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline

linear = LinearRegression(featuresCol="features", labelCol="medv")
pipeline = Pipeline(stages=[vector_assembler, linear])
param_grid = ParamGridBuilder() \
    .addGrid(linear.elasticNetParam, [0.01, 0.02, 0.05]) \
    .addGrid(linear.solver, ['normal', 'l-bfgs']) \
    .addGrid(linear.regParam, [0.4, 0.5, 0.6]).build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=param_grid,
                          evaluator=evaluator,
                          numFolds=10)

optimized_model = crossval.fit(housing_df)


ModuleNotFoundError: No module named 'pyspark'

In [128]:
[(k.name, v) for (k, v) in optimized_model.bestModel.stages[1].extractParamMap().items()]


[('epsilon', 1.35),
 ('featuresCol', 'features'),
 ('predictionCol', 'prediction'),
 ('loss', 'squaredError'),
 ('elasticNetParam', 0.02),
 ('regParam', 0.6),
 ('maxIter', 100),
 ('labelCol', 'medv'),
 ('tol', 1e-06),
 ('standardization', True),
 ('aggregationDepth', 2),
 ('fitIntercept', True),
 ('solver', 'l-bfgs')]

In [115]:
optimized_model.avgMetrics


[0.60228046689935,
 0.6022857524897973,
 0.6023834177393386,
 0.6023887410814059,
 0.6026432049338516,
 0.6026481973755449,
 0.6030001230672197,
 0.6030052621812348,
 0.6031040135770328,
 0.6031091439263058,
 0.6032064249778748,
 0.6032086198971643,
 0.60355796933488,
 0.6035626800336215,
 0.6036587578463659,
 0.603663431370495,
 0.6034106428627964,
 0.6034118340373834]

In [117]:
_, evaluation_df = housing_df.randomSplit([0.8, 0.2], seed=17)
evaluator.evaluate(optimized_model.transform(evaluation_df))


0.7202817971205354